In [1]:
!pip install mne
!pip install numpy matplotlib pywavelets mne
!pip install numpy mne matplotlib scipy
!pip install pyedflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 15.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import pywt
from pywt import wavedec
import os
import pyedflib

In [4]:
# 데이터와 저장 경로 설정
raw_path = "/content/drive/MyDrive/데캡디(딥슬립딥드림)공유문서함/DATA/sleep_data/recordings/"
cleaned_path = "/content/drive/MyDrive/데캡디(딥슬립딥드림)공유문서함/DATA/Pre_rev/"
if not os.path.exists(cleaned_path):
    os.makedirs(cleaned_path)


def save_as_edf(raw, filename):
    with pyedflib.EdfWriter(filename,
                            n_channels=len(raw.ch_names),
                            file_type=pyedflib.FILETYPE_EDFPLUS) as writer:
        channel_info = []
        data = []
        for i, ch in enumerate(raw.ch_names):
            ch_dict = {'label': ch, 'dimension': 'uV', 'sample_rate': int(raw.info['sfreq']), 'physical_min': raw._data[i].min(), 'physical_max': raw._data[i].max(), 'digital_min': 0, 'digital_max': 9, 'transducer': '', 'prefilter': ''}
            channel_info.append(ch_dict)
            data.append(raw._data[i])

        writer.setSignalHeaders(channel_info)
        writer.writeSamples(data)

def preprocess_psg(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True)
    new_sampling_rate = 100

    # 채널 타입을 설정하기 위한 사전 정의
    channel_types = {
        raw.ch_names[0]: 'eeg',  # 인덱스 0
        raw.ch_names[1]: 'eeg',  # 인덱스 1
        raw.ch_names[2]: 'eeg',  # 인덱스 2
        raw.ch_names[3]: 'eeg',  # 인덱스 3
        raw.ch_names[4]: 'emg',  # 인덱스 4, EMG chin
        raw.ch_names[5]: 'eog',  # 인덱스 5, EOG
        raw.ch_names[6]: 'eog',  # 인덱스 6, EOG
        raw.ch_names[7]: 'ecg'   # 인덱스 7, ECG
    }

    # 채널 타입을 설정
    raw.set_channel_types(channel_types)

    # filter
    raw.filter(l_freq=15, h_freq=None, picks='EMG chin')  # 15Hz 이상으로 밴드패스 필터링
    raw.filter(0.2, 35, picks = ['EEG F4-M1','EEG C4-M1','EEG O2-M1','EEG C3-M2', 'EOG E1-M2', 'EOG E2-M2'])
    raw.filter(0.5, 50, picks='ECG')

    # notch filter
    raw.notch_filter(50)

    # 함수 정의
    # Threshold
    # 중앙절대편차(MAD)를 이용한 잡음 표준편차 추정
    def mad(data, axis=None):
        return np.median(np.abs(data - np.median(data, axis)), axis)

    # Universal 임계값 계산
    def universal_threshold(data):
        sigma = mad(data) / 0.6745  # 잡음의 표준편차 추정
        return sigma * np.sqrt(2 * np.log(len(data)))

    # Minimax 임계값 계산
    def minimax_threshold(data, sigma=None):
        n = len(data)
        if sigma is None:
            sigma = mad(data) / 0.6745  # MAD를 통한 잡음 추정
        return sigma * np.sqrt(2 * np.log(n) * np.log(n) / n)

    # Hybrid Threshold 계산
    def hybrid_threshold(data):
        uni_thresh = universal_threshold(data)
        mini_thresh = minimax_threshold(data)
        # 가중 평균 계산: Universal에 70%, Minimax에 30%
        return 0.7 * uni_thresh + 0.3 * mini_thresh  # 평균 사용

    # eeg denosing
    def wavelet_denoising_eeg(data, wavelet_name='db2', level=2):
        wavelet = pywt.Wavelet(wavelet_name)
        coeff = pywt.wavedec(data, wavelet, level=level)
        sigma = (1 / 0.6745) * mad(coeff[-2])
        uthresh = sigma * np.sqrt(2 * np.log(data.shape[1]))
        coeff[1:] = [pywt.threshold(i, value=uthresh, mode='soft') for i in coeff[1:]]
        reconstructed_data = pywt.waverec(coeff, wavelet)

        return reconstructed_data

    # emg donosing
    # 웨이블릿 노이즈 제거 함수
    def wavelet_denoising_emg(data, wavelet):
        wavelet = pywt.Wavelet(wavelet)  # 웨이블릿 객체 생성
        max_level = 2  # 최대 분해 레벨 계산
        coeff = pywt.wavedec(data, wavelet, mode="reflect", level=max_level)  # 웨이블릿 분해
        # Hybrid Threshold 적용
        for i in range(1, len(coeff)):
            hybrid_thresh = hybrid_threshold(coeff[i])
            coeff[i] = pywt.threshold(coeff[i], value=hybrid_thresh, mode='soft')  # 소프트 임계값 사용
        reconstructed_signal_emg = pywt.waverec(coeff, wavelet, mode='reflect')  # 웨이블릿 재구성
        return reconstructed_signal_emg

    # eog denosing
    # Function to perform wavelet denoising
    def wavelet_denoise_eog(data, wavelet, level):
        # Decompose to get the wavelet coefficients
        coeff = pywt.wavedec(data, wavelet, mode="per", level=level)
        sigma = (1/0.6745) * mad(coeff[-level])

        # Threshold for each detail coefficient
        uthresh = sigma * np.sqrt(2*np.log(len(data)))
        coeff[1:] = (pywt.threshold(i, value=uthresh, mode='soft') for i in coeff[1:])

        # Reconstruct the signal using the thresholded coefficients
        return pywt.waverec(coeff, wavelet, mode='per')

    # ecg denoising
    # remove baseline wander
    def remove_baseline_wander(ecg_signal, wavelet='db4', level=2):
        # Wavelet decomposition
        coeffs = pywt.wavedec(ecg_signal, wavelet, mode="reflect", level=level)
        # 잡음의 표준편차(잡읍 추정치)
        sigma = (1/0.6745) * mad(coeffs[-level])
        # 유니버셜 Threshold계산
        uthresh = sigma * np.sqrt(2 * np.log(len(ecg_signal)))
        # Thresholding and filtering approximation coefficients
        for i in range(1, len(coeffs)):
            coeffs[i] = pywt.threshold(coeffs[i], value=uthresh, mode='soft')
        # Wavelet reconstruction
        clean_ecg_signal = pywt.waverec(coeffs, wavelet, mode='reflect')

        return clean_ecg_signal



# EEG
    eeg_data = raw.copy().get_data(picks=[0, 1, 2, 3])
    denoised_eeg = wavelet_denoising_eeg(eeg_data)
    raw._data[0:4, :] = denoised_eeg

# EMG
    # 'EMG chin' 채널 데이터만 가져오기
    emg_data = raw.get_data()[4]  # 채널 인덱스 4는 예시로 사용된 것이며, 실제 채널 인덱스를 확인해야 합니다.
    # Applying wavelet denoising
    clean_emg_data = wavelet_denoising_emg(emg_data, 'db4')
    # 전처리된 데이터를 원본 Raw 데이터에 업데이트
    raw._data[raw.ch_names.index('EMG chin'), :] = clean_emg_data



#EOG
    eog_data0 = raw.copy().get_data(picks=[5]).squeeze()
    # Perform denoising
    eog_data_dn = wavelet_denoise_eog(eog_data0, 'db4', level=4)
    #fit in data
    raw._data[5] = eog_data_dn[np.newaxis, :]
    eog_data1 = raw.copy().get_data(picks=[6]).squeeze()
    # Perform denoising
    eog_data_dn = wavelet_denoise_eog(eog_data1, 'db4', level=4)
    #fit in data
    raw._data[6] = eog_data_dn[np.newaxis, :]


# ECG
    raw_ecg = raw.copy().pick_channels(['ECG'])
    sfreq = raw_ecg.info['sfreq']  # 샘플링 주파수
    # get_data()로 데이터 가져오기
    filtered_ecg_data = raw.get_data()[7].squeeze()

    clean_ecg_data = remove_baseline_wander(filtered_ecg_data)

    # 전처리된 데이터를 원본 Raw 데이터에 업데이트
    raw._data[raw.ch_names.index('ECG'), :] = clean_ecg_data


    # 리샘플링
    raw.resample(new_sampling_rate)

    # 파일 저장
    new_file_path = os.path.join(cleaned_path, "pre_" + os.path.basename(file_path))
    save_as_edf(raw, new_file_path)



# 모든 파일에 대해 전처리 실행
for i in range(1, 155):  # 1부터 154까지 반복
    idx = str(i).zfill(3)
    file = os.path.join(raw_path, "SN" + idx + ".edf")
    if os.path.exists(file):
        preprocess_psg(file)
        print(f"Processed and saved: pre_SN{idx}.edf")
    else:
        print(f"File SN{idx}.edf not found")

Extracting EDF parameters from /content/drive/MyDrive/데캡디(딥슬립딥드림)공유문서함/DATA/sleep_data/recordings/SN001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-4-e9d1f89b360a>:23: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(file_path, preload=True)
<ipython-input-4-e9d1f89b360a>:23: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(file_path, preload=True)


Reading 0 ... 6566399  =      0.000 ... 25649.996 secs...
Filtering a subset of channels. The highpass and lowpass values in the measurement info will not be updated.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 15 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 15.00
- Lower transition bandwidth: 3.75 Hz (-6 dB cutoff frequency: 13.12 Hz)
- Filter length: 227 samples (0.887 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.2 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.20
- Lower transition bandwidth: 0.20 Hz

/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:124: UserWarning: Physical minimum for channel 0 (EEG F4-M1) is -0.0008763216971685162, which has 22 chars, however, EDF+ can only save 8 chars, will be truncated to -0.00087, some loss of precision is to be expected
  warnings.warn('Physical minimum for channel {} ({}) is {}, which has {} chars, '\
/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:131: UserWarning: Physical minimum for channel 0 (EEG F4-M1) is 0.0009430826363069539, which has 21 chars, however, EDF+ can only save 8 chars, will be truncated to 0.000943, some loss of precision is to be expected.
  warnings.warn('Physical minimum for channel {} ({}) is {}, which has {} chars, '\
/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:124: UserWarning: Physical minimum for channel 1 (EEG C4-M1) is -0.0008618661114035722, which has 22 chars, however, EDF+ can only save 8 chars, will be truncated to -0.00086, some loss of precision is to be expe

Processed and saved: pre_SN001.edf
Extracting EDF parameters from /content/drive/MyDrive/데캡디(딥슬립딥드림)공유문서함/DATA/sleep_data/recordings/SN002.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 6577407  =      0.000 ... 25692.996 secs...


...

Processed and saved: pre_SN153.edf
Extracting EDF parameters from /content/drive/MyDrive/데캡디(딥슬립딥드림)공유문서함/DATA/sleep_data/recordings/SN154.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7708415  =      0.000 ... 30110.996 secs...


<ipython-input-4-e9d1f89b360a>:23: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(file_path, preload=True)
<ipython-input-4-e9d1f89b360a>:23: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(file_path, preload=True)


Filtering a subset of channels. The highpass and lowpass values in the measurement info will not be updated.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 15 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 15.00
- Lower transition bandwidth: 3.75 Hz (-6 dB cutoff frequency: 13.12 Hz)
- Filter length: 227 samples (0.887 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.2 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.20
- Lower transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.10 Hz)
- Upper passband edge: 

/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:124: UserWarning: Physical minimum for channel 0 (EEG F4-M1) is -0.0008100655617543387, which has 22 chars, however, EDF+ can only save 8 chars, will be truncated to -0.00081, some loss of precision is to be expected
  warnings.warn('Physical minimum for channel {} ({}) is {}, which has {} chars, '\
/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:131: UserWarning: Physical minimum for channel 0 (EEG F4-M1) is 0.0008521076032829317, which has 21 chars, however, EDF+ can only save 8 chars, will be truncated to 0.000852, some loss of precision is to be expected.
  warnings.warn('Physical minimum for channel {} ({}) is {}, which has {} chars, '\
/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:124: UserWarning: Physical minimum for channel 1 (EEG C4-M1) is -0.000587758838594729, which has 21 chars, however, EDF+ can only save 8 chars, will be truncated to -0.00058, some loss of precision is to be expec

Processed and saved: pre_SN154.edf
